In [1]:
import json
from collections import deque

In [2]:
# Load the JSON representations of the domain and problem
with open('domain.json', 'r') as domain_file:
    domain_data = json.load(domain_file)

with open('task1.json', 'r') as problem_file:
    problem_data = json.load(problem_file)

In [3]:
# Extract actions from the domain
actions = domain_data.get("actions", [])

In [4]:
# Extract objects from the task file 
objects = problem_data.get("objects", [])

In [5]:
# Generate all possible combinations of actions and objects
combinations = []

for action in actions:
    for obj1 in objects:
        if len(action["parameters"]) == 1:
            # If the action takes one object, create a combination
            combinations.append(f"('{action['name']}', '{obj1}')")
        else:
            for obj2 in objects:
                if obj1 != obj2:
                    # If the action takes two different objects, create a combination
                    combinations.append(f"('{action['name']}', '{obj1}', '{obj2}')")

In [6]:
# Print the output
for combination in combinations:
    print(combination)

('pick-up', 'D')
('pick-up', 'B')
('pick-up', 'A')
('pick-up', 'C')
('put-down', 'D')
('put-down', 'B')
('put-down', 'A')
('put-down', 'C')
('stack', 'D', 'B')
('stack', 'D', 'A')
('stack', 'D', 'C')
('stack', 'B', 'D')
('stack', 'B', 'A')
('stack', 'B', 'C')
('stack', 'A', 'D')
('stack', 'A', 'B')
('stack', 'A', 'C')
('stack', 'C', 'D')
('stack', 'C', 'B')
('stack', 'C', 'A')
('unstack', 'D', 'B')
('unstack', 'D', 'A')
('unstack', 'D', 'C')
('unstack', 'B', 'D')
('unstack', 'B', 'A')
('unstack', 'B', 'C')
('unstack', 'A', 'D')
('unstack', 'A', 'B')
('unstack', 'A', 'C')
('unstack', 'C', 'D')
('unstack', 'C', 'B')
('unstack', 'C', 'A')


<h1>method 1 : </h1>

<h3>я загрузил исходную ситуацию, целевую ситуацию и набор действий из файлов JSON.
Использовал BFS для поиска решения, исследуя все возможные последовательности действий для достижения целевой ситуации из исходной ситуации </h3>

In [7]:
# Define a function for breadth-first search
def bfs(domain_file, problem_file):
    # Extract initial and goal states as lists of strings
    initial_state = [str(item) for item in problem_data.get("init", [])]
    goal_state = [str(item) for item in problem_data.get("and", [])]

    # Define a queue for BFS
    queue = deque()
    visited_states = set()

    # Initialize the queue with the initial state
    initial_state_str = json.dumps(initial_state, sort_keys=True)
    queue.append(initial_state_str)
    visited_states.add(initial_state_str)

    while queue:
        current_state = queue.popleft()

        if current_state == json.dumps(goal_state, sort_keys=True):
            return "Solution found!"

        current_state_dict = json.loads(current_state)
        for action in actions:
            # Try applying each action to the current state
            new_state = apply_action(action, current_state_dict)
            new_state_str = json.dumps(new_state, sort_keys=True)
            
            if new_state_str not in visited_states:
                visited_states.add(new_state_str)
                queue.append(new_state_str)

    return "No solution found."

In [8]:
# Define a function to apply an action to the state
def apply_action(action, state):
    new_state = state.copy()
    # Implement action effects here based on PDDL-style effects
    action_name = action["name"]
    parameters = action["parameters"]

    # Search for the action in the state and update the state
    for key in new_state:
        if action_name in key:
            if "not" in key:
                # Remove the negative effect
                new_state.remove(key)
            else:
                # Replace the positive effect
                new_state.remove(key)
                new_state.append(action_name)
    return new_state

In [9]:
# Extract initial and goal states from the JSON data
initial_state = [str(item) for item in problem_data.get("init", [])]
goal_state = [str(item) for item in problem_data.get("and", [])]
result = bfs(domain_data, problem_data)
print(result)

No solution found.


<h1>method 2: </h1>

<h3>я вручную определил исходные ситуации, цели и список комбинаций действий,
затем я выполнил действия по исходным ситуациям и проверил, достигнуты ли цели</h3>

In [15]:
# Define the initial state as a list of strings
initial_state = [
    "CLEAR C",
    "CLEAR A",
    "CLEAR B",
    "CLEAR D",
    "ONTABLE C",
    "ONTABLE A",
    "ONTABLE B",
    "ONTABLE D",
    "HANDEMPTY"
]

In [23]:
# Define the list of goals as a list of strings
goals = [
    "ON D C",
    "ON C B",
    "ON B A"
]

In [17]:
combinations = [
    ('pick-up', 'D'),
    ('stack', 'D', 'C'),
    ('pick-up', 'C'),
    ('stack', 'C', 'B'),
    ('pick-up', 'B'),
    ('stack', 'B', 'A')
]

In [18]:
# Function to perform an action on a state
def perform_action(state, action):
    if action[0] == 'pick-up':
        if f"CLEAR {action[1]}" in state and f"ONTABLE {action[1]}" in state and "HANDEMPTY" in state:
            state.remove(f"CLEAR {action[1]}")
            state.remove(f"ONTABLE {action[1]}")
            state.remove("HANDEMPTY")
            state.append(f"HOLDING {action[1]}")
    elif action[0] == 'put-down':
        if f"HOLDING {action[1]}" in state:
            state.remove(f"HOLDING {action[1]}")
            state.append(f"CLEAR {action[1]}")
            state.append(f"ONTABLE {action[1]}")
            state.append("HANDEMPTY")
    elif action[0] == 'stack':
        if f"HOLDING {action[1]}" in state and f"CLEAR {action[2]}" in state:
            state.remove(f"HOLDING {action[1]}")
            state.remove(f"CLEAR {action[2]}")
            state.append(f"CLEAR {action[1]}")
            state.append(f"ON {action[1]} {action[2]}")
            state.append("HANDEMPTY")
    return state

In [19]:
# Initialize the state with the initial state
current_state = list(initial_state)

In [20]:
# Function to check if all goals are satisfied
def are_goals_satisfied(state, goals):
    return all(goal in state for goal in goals)

In [21]:
# DFS to find all solutions
def find_all_solutions(state, goals, path):
    if are_goals_satisfied(state, goals):
        yield path  # Return the current solution

    for combination in combinations:
        new_state = perform_action(list(state), combination)
        if new_state not in path:
            new_path = path + [new_state]
            # Recursively explore the next step
            yield from find_all_solutions(new_state, goals, new_path)

In [25]:
# Find and print all solutions
initial_state_str = list(initial_state)
solutions = find_all_solutions(initial_state_str, goals, [initial_state_str])

for solution in solutions:
    print(f"Solutions:")
    for state in solution:
        print(state)

Solutions:
['CLEAR C', 'CLEAR A', 'CLEAR B', 'CLEAR D', 'ONTABLE C', 'ONTABLE A', 'ONTABLE B', 'ONTABLE D', 'HANDEMPTY']
['CLEAR C', 'CLEAR A', 'CLEAR D', 'ONTABLE C', 'ONTABLE A', 'ONTABLE D', 'HOLDING B']
['CLEAR C', 'CLEAR D', 'ONTABLE C', 'ONTABLE A', 'ONTABLE D', 'CLEAR B', 'ON B A', 'HANDEMPTY']
['CLEAR D', 'ONTABLE A', 'ONTABLE D', 'CLEAR B', 'ON B A', 'HOLDING C']
['CLEAR D', 'ONTABLE A', 'ONTABLE D', 'ON B A', 'CLEAR C', 'ON C B', 'HANDEMPTY']
['ONTABLE A', 'ON B A', 'CLEAR C', 'ON C B', 'HOLDING D']
['ONTABLE A', 'ON B A', 'ON C B', 'CLEAR D', 'ON D C', 'HANDEMPTY']
